# **Generative AI Video Insights Generator Using Google- Gemini and Streamlit**

## **Overview**

This project is a Streamlit-based web application that allows users to upload a video file and analyze its content using Google's Gemini API. The application leverages the capabilities of large language models to extract insights from video content and have various uses, some of which are mentioned below:

**1. Research and Academic Projects**

*   **Lecture Summarization:** Quickly extract key points from academic lectures or research presentations.
*   **Qualitative Research:** Analyze interviews or focus group discussions for patterns and themes.
*   **Experiment Documentation:** Review and summarize laboratory experiment recordings.
    

**2. Business and Market Analysis**


*   **Competitor Video Analysis:** Extract insights from promotional videos, product launches, or advertisements.
*   **Customer Feedback:** Analyze customer testimonial videos to identify trends and areas for improvement.



## **Execution Flow**

To help you better understand how this project works, here’s a breakdown of the key steps involved in its execution:

1. Application starts with __init__() to set up environment and API configuration.

2. Streamlit interface is rendered with **app()**.

3. User uploads a video and inputs a prompt.

4. Video file is saved locally using **save_uploaded_file()**.

5. Video is displayed on the Streamlit app.

6. **get_insights()** uploads the video to Gemini API and sends a prompt for analysis.

7. Insights from the video are displayed to the user.

8. Temporary video file is deleted from the local system.

## **Project Structure**

In [ ]:
project_folder/
|-- medias/               # Directory to store uploaded media files
|-- app.py                # Main application code
|-- .env                  # Environment variables file containing GEMINI_API_KEY

## **Key Components**

### **1. Initialization**

This block initializes the environment and API configuration.

*   Imports necessary libraries.
*   Creates a folder medias/ to store uploaded videos if it doesn't exist.
*   Loads the API key from the .env file using dotenv.
*   Configures the genai client with the API key.



In [ ]:
import os
import time
import google.generativeai as genai
import streamlit as st
from dotenv import load_dotenv

MEDIA_FOLDER = 'medias'

def __init__():
    if not os.path.exists(MEDIA_FOLDER):
        os.makedirs(MEDIA_FOLDER)

    load_dotenv()
    api_key = os.getenv("GEMINI_API_KEY")
    genai.configure(api_key=api_key)

### **2. Saving Uploaded Files**

This block handles file uploads. It:

*   Accepts an uploaded file object from the user.
*   Saves the file into the medias/ directory with its original filename.
*   Returns the path of the saved file.


In [ ]:
def save_uploaded_file(uploaded_file):
    file_path = os.path.join(MEDIA_FOLDER, uploaded_file.name)
    with open(file_path, 'wb') as f:
        f.write(uploaded_file.read())
    return file_path

### **3. Extracting Video Insights**

This block interacts with the Gemini API to process the uploaded video and generate insights. It:

*   Uploads the video file to the Gemini API.
*   Polls the API until the video processing is complete.
*   Sends the video and user-specified prompt to a generative model for analysis.
*   Displays the insights in the Streamlit interface.
*   Deletes the video from the API storage after processing.


In [ ]:
def get_insights(video_path, prompt):
    st.write(f"Processing video: {video_path}")

    st.write(f"Uploading file...")
    video_file = genai.upload_file(path=video_path)
    st.write(f"Completed upload: {video_file.uri}")

    while video_file.state.name == "PROCESSING":
        st.write('Waiting for video to be processed.')
        time.sleep(10)
        video_file = genai.get_file(video_file.name)

    if video_file.state.name == "FAILED":
        raise ValueError(video_file.state.name)

    model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

    st.write("Making LLM inference request...")
    response = model.generate_content([prompt, video_file],
                                      request_options={"timeout": 600})
    st.write(f'Video processing complete')
    st.subheader("Insights")
    st.write(response.text)
    genai.delete_file(video_file.name)

### **4. Streamlit Application**

This block builds the user interface for the application. It:

*   Displays a file uploader and a text input area for the prompt.
*   Saves the uploaded video file locally.
*   Displays the uploaded video within the app.
*   Calls **get_insights()** to analyze the video content.
*   Deletes the local video file after processing.


In [ ]:
def app():
    st.title("Video Insights Generator")

    uploaded_file = st.file_uploader("Upload a video file", type=["mp4", "avi", "mov", "mkv"])

    prompt = st.text_area("Enter your prompt for video analysis")

    if uploaded_file is not None and prompt:
        file_path = save_uploaded_file(uploaded_file)
        st.video(file_path)
        get_insights(file_path, prompt)

        if os.path.exists(file_path):
            os.remove(file_path)

### **5. Running the Application**

This block triggers the execution of the program. It:

*   Calls __init__() method to set up the environment and API configuration.
*   Calls **app()** method to launch the Streamlit application.


In [ ]:
__init__()
app()